# 사전 작업
## Conda 환경 생성 및 패키지 설치
터미널에서 새로운 conda 환경 생성(anaconda prompt에서)

In [ ]:
conda create -n paddle_env python=3.9 -y
conda activate paddle_env

## 필요한 라이브러리 설치
Conda 환경에서 필요한 모든 라이브러리를 설치

In [ ]:
# PaddlePaddle 및 관련 패키지 설치
conda install -c conda-forge paddlepaddle-gpu
conda install -c conda-forge paddlepaddle
pip install paddleocr

# LabelImg 설치
pip install labelImg

# Scikit-learn 및 기타 필요한 패키지 설치
conda install -c conda-forge scikit-learn
conda install -c conda-forge scikit-image
conda install -c conda-forge threadpoolctl

# 폰트 설치 (필요하면 하셈)
conda install -c conda-forge font-ttf-nanum

# 라이브러리 설치

In [1]:
%pip install paddleocr
%pip install paddlepaddle-gpu
!apt-get update -qq
!apt-get install -qq fonts-nanum
!fc-list | grep "NanumGothic"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'fc-list'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


# VScode에서 실행할 스크립트

## 전체 코드

In [1]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from matplotlib import pyplot as plt

In [2]:
def plt_imshow(title='image', img=None, figsize=(8, 5)):
    plt.figure(figsize=figsize)

    if type(img) is str:
        img = cv2.imread(img)

    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)

            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])

        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

def put_text(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_converted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_converted)

    # Use the installed NanumGothic font
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

    try:
        image_font = ImageFont.truetype(font_path, font_size)
    except IOError:
        image_font = ImageFont.load_default()

    draw = ImageDraw.Draw(image)

    try:
        draw.text((x, y), text, font=image_font, fill=color)
    except UnicodeEncodeError:
        # Fallback to default font for unsupported characters
        fallback_font = ImageFont.load_default()
        draw.text((x, y), text, font=fallback_font, fill=color)

    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

    return opencv_image

from paddleocr import PaddleOCR, draw_ocr
from paddleocr.paddleocr import MODEL_URLS

class MyPaddleOCR:
    def __init__(self, lang: str = "korean", **kwargs):
        self.lang = lang
        self._ocr = PaddleOCR(lang="korean", use_gpu=True, use_angle_cls = True, use_space_char = True,)
        self.img_path = None
        self.ocr_result = {}

    def get_available_langs(self):
        langs_info = []

        for idx, model_name in enumerate(list(MODEL_URLS['OCR'].keys())):
            for lang in list(MODEL_URLS['OCR'][model_name]['rec'].keys()):
                if lang not in langs_info:
                    langs_info.append(lang)

        print('Available Language : {}'.format(langs_info))

    def get_available_models(self):
        model_info = {}

        for idx, model_name in enumerate(list(MODEL_URLS['OCR'].keys())):
            model_info[model_name] = list(MODEL_URLS['OCR'][model_name]['rec'].keys())
            print('#{} Model Vesion : [{}] - Language : {}'.format(idx+1, model_name, list(MODEL_URLS['OCR'][model_name]['rec'].keys())))

    def get_ocr_result(self):
        return self.ocr_result

    def get_img_path(self):
        return self.img_path

    def show_img(self):
        plt_imshow(img=self.img_path)

    def run_ocr(self, img_path: str, debug: bool = False, raw=False):
        self.img_path = img_path
        ocr_text = []
        result = self._ocr.ocr(img_path, cls=False)
        self.ocr_result = result[0]

        if self.ocr_result:
            for r in result[0]:
                ocr_text.append(r[1][0])
        else:
            ocr_text = "No text detected."

        if debug:
            self.show_img_with_ocr()
        if raw:
          return ocr_text
        else:
          return filter_food(ocr_text)

    def show_img_with_ocr(self):
        img = cv2.imread(self.img_path)
        roi_img = img.copy()

        for text_result in self.ocr_result:
            text = text_result[1][0]
            tlX = int(text_result[0][0][0])
            tlY = int(text_result[0][0][1])
            trX = int(text_result[0][1][0])
            trY = int(text_result[0][1][1])
            brX = int(text_result[0][2][0])
            brY = int(text_result[0][2][1])
            blX = int(text_result[0][3][0])
            blY = int(text_result[0][3][1])

            pts = ((tlX, tlY), (trX, trY), (brX, brY), (blX, blY))

            topLeft = pts[0]
            topRight = pts[1]
            bottomRight = pts[2]
            bottomLeft = pts[3]

            cv2.line(roi_img, topLeft, topRight, (0, 255, 0), 2)
            cv2.line(roi_img, topRight, bottomRight, (0, 255, 0), 2)
            cv2.line(roi_img, bottomRight, bottomLeft, (0, 255, 0), 2)
            cv2.line(roi_img, bottomLeft, topLeft, (0, 255, 0), 2)
            roi_img = put_text(roi_img, text, topLeft[0], topLeft[1] - 20, font_size=15)

        plt_imshow(["Original", "ROI"], [img, roi_img], figsize=(16, 10))

# Example usage:
ocr = MyPaddleOCR()
res = ocr.run_ocr('C:/Users/user/Desktop/coupang/images/rec1.jpg', debug=False, raw=True)


[2024/06/28 17:34:19] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\user/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\user/.paddleocr/whl\\rec\\korean\\korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num

In [3]:
res

['CoUpang',
 'eats',
 '객용]',
 '1VX4BA',
 '[수저포크0]',
 '메뉴',
 '수량',
 '금액',
 '로제이닭강정',
 '세트',
 '22000',
 '허니갈릭',
 '달콤간장',
 '닭강경',
 '1500',
 '주',
 '2단계',
 '보통맛',
 '0',
 '후루룩누틀밀떡',
 '+',
 '치킨무0주세요',
 '_',
 '3',
 '500',
 '분모자추가',
 '2009',
 '수',
 '1 500',
 '유부추가',
 '4개',
 '5개',
 '1500',
 '메추리알',
 '30',
 '000',
 '주문금액',
 '2',
 '700',
 '배달비',
 '-2',
 '000',
 '할인금액',
 '-1700',
 '배달비할인',
 '29',
 '000',
 '카드결제',
 '29000',
 '총결제금액',
 '거래일시:',
 '2024-06-27',
 '12:33:30',
 '주문매장:',
 '태리로제떡볶이&닭강정',
 '교대점',
 '결제방식',
 '쿠페이',
 '선결제',
 '완료',
 '쿠팡이츠',
 '고객센터',
 '1670-9827']

In [4]:
import re
def extract_food_names(data):
    food_names = []
    i = data.index('금액') + 1
    current_food = []

    while i < len(data):
        item = data[i]
        if re.match(r'\d+|\+|_', item):
            if current_food:
                food_names.append(' '.join(current_food))
                current_food = []
        if not re.match(r'\d+|\+|_', item):
            current_food.append(item)
        i += 1

    if current_food:
        food_names.append(' '.join(current_food))

    return food_names

In [5]:
def extract_food_names_until_keywords(data):
    food_names = []
    keywords = ['판매액', '주문금액', '물품']
    i = data.index('금액') + 1
    current_food = []

    while i < len(data):
        item = data[i]
        if item in keywords:
            break
        if re.match(r'\d+|\+|_', item):
            if current_food:
                food_names.append(' '.join(current_food))
                current_food = []
        if not re.match(r'\d+|\+|_', item):
            current_food.append(item)
        i += 1

    if current_food:
        food_names.append(' '.join(current_food))

    return food_names

# Example usage:
ocr = MyPaddleOCR()
res = ocr.run_ocr('C:/Users/user/Desktop/coupang/images/rec1.jpg', debug=False, raw=True)

# 음식 이름 추출 (특정 키워드 전까지만)
food_names = extract_food_names_until_keywords(res)
print(food_names)


[2024/06/28 17:34:39] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\user/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\user/.paddleocr/whl\\rec\\korean\\korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num

# 미완성 코드

In [18]:
def extract_food_names_until_keywords(data):
    food_names = []
    keywords = ['판매액', '주문금액', '물품']
    current_food = []

    for item in data:
        if item in keywords:
            break
        if re.match(r'\d+|\+|_', item):
            if current_food:
                food_names.append(' '.join(current_food))
                current_food = []
        if not re.match(r'\d+|\+|_', item):
            current_food.append(item)

    if current_food:
        food_names.append(' '.join(current_food))

    return food_names

# Example usage:
ocr = MyPaddleOCR()
res = ocr.run_ocr('C:/Users/user/Desktop/coupang/images/rec1.jpg', debug=False, raw=True)

# 음식 이름 추출 (특정 키워드 전까지만)
food_names = extract_food_names_until_keywords(res)
print(food_names)

[2024/06/28 17:26:12] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\user/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\user/.paddleocr/whl\\rec\\korean\\korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num